In [144]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn import preprocessing
np.random.seed(42)

In [145]:
X_train_in = pd.read_csv("X_train.csv", index_col=0)
y_train_in = pd.read_csv("y_train.csv",index_col=0)
X_val = pd.read_csv("X_test.csv", index_col=0)
st_info = pd.read_csv("studs_info.csv")

In [146]:
X_all = X_train_in.append(X_val, ignore_index=True)

In [147]:
X_train_in.head(1)

,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА
51081,175711,38.03.02 Менеджмент,2018-2019,2.0,4.0,Экзамен,Маркетинг


In [148]:
student_level = []
big_group = []
small_group = []

for i in X_train_in['НАПРАВЛЕНИЕ']:
    student_level.append(i.split('.')[1])
    big_group.append(i.split('.')[0])
    small_group.append(i.split('.')[2][:2])
    
X_train_in['STD_LVL'] = student_level
X_train_in['big_group'] = big_group
X_train_in['small_group'] = small_group

In [149]:
# {'Дифференцированный зачет',5
#  'Зачет',1
#  'Курсовая работа',3
#  'Курсовой проект',4
#  'Экзамен' 2}
X_train_in['АТТЕСТАЦИЯ'].replace(['Дифференцированный зачет','Зачет', 'Курсовая работа', 'Курсовой проект','Экзамен'],[5,1,3,4,2],inplace=True)

In [150]:
st_info = st_info.drop_duplicates(subset=['STD_ID'], keep="last")
st_info["Образование"].fillna("СР", inplace = True)
st_info["Что именно закончил"].fillna("-", inplace = True)
mage = []
bablo = []
for i in st_info["Дата выдачи"]:
    age = pd.to_datetime(i, format='%Y-%m-%d %H:%M:%S', errors='ignore')
    if isinstance(age, pd._libs.tslibs.timestamps.Timestamp):
        mage.append(age.today().year - age.year + 18)
    else:
        mage.append(18)
for i in st_info["Категория обучения"]:
    bablo.append(i == "БП")
st_info["age"] = mage
st_info["bablo"] = bablo
le = preprocessing.LabelEncoder()
le.fit(st_info.Образование)
st_info["learn"] = le.transform(st_info["Образование"])

st_info['Пол'].replace(['М','Ж'],[0, 1],inplace=True)
le = preprocessing.LabelEncoder()
le.fit(st_info["Что именно закончил"])
st_info["school"] = le.transform(st_info["Что именно закончил"])

le = preprocessing.LabelEncoder()
le.fit(X_all.ГОД)
X_train_in.loc[:,"exam_year"] = le.transform(X_train_in.loc[:,"ГОД"])
X_val.loc[:,"exam_year"] = le.transform(X_val.loc[:,"ГОД"])

In [151]:
le = preprocessing.LabelEncoder()

le.fit(X_all.ДИСЦИПЛИНА)

X_train_in.loc[:,"ДИСЦИПЛИНА"] = le.transform(X_train_in.loc[:,"ДИСЦИПЛИНА"])
X_val.loc[:,"ДИСЦИПЛИНА"] = le.transform(X_val.loc[:,"ДИСЦИПЛИНА"])

In [152]:
X_train_in["mark"] = y_train_in["mark"]

In [153]:
X_train_in

,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА,STD_LVL,big_group,small_group,exam_year,mark
51081,175711,38.03.02 Менеджмент,2018-2019,2.0,4.0,2,408,03,38,02,5,5
72921,100647,31.05.01 Лечебное дело,2015-2016,3.0,5.0,2,946,05,31,01,2,5
96438,199311,31.05.03 Стоматология,2019-2020,1.0,1.0,2,1326,05,31,03,6,5
273044,162002,38.03.02 Менеджмент,2017-2018,2.0,3.0,2,1100,03,38,02,4,4
371309,108384,31.05.01 Лечебное дело,2018-2019,5.0,9.0,2,713,05,31,01,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...
224856,176141,40.03.01 Юриспруденция,2017-2018,1.0,2.0,3,364,03,40,01,4,3
260998,182794,31.05.01 Лечебное дело,2018-2019,2.0,4.0,2,556,05,31,01,5,3
57245,162391,31.05.01 Лечебное дело,2016-2017,1.0,2.0,5,228,05,31,01,3,4
218529,202479,38.03.01 Экономика,2017-2018,1.0,1.0,5,238,03,38,01,4,4


In [154]:
#School rate
schools = []
school_rates = []
g_school_rates = []
for school in st_info["school"].unique():
    schools.append(school)
    rate = 0
    count = 0
    global_rate = 0
    global_count = 0
#     print(st_info[st_info["Что именно закончил"] == school].STD_ID)
#     break
    for student in st_info[st_info["school"] == school].STD_ID:
        for mark in X_train_in[X_train_in.STD_ID == student].iterrows():
            mark = mark[1]
            global_rate += mark.mark
            global_count += 1
            if mark.КУРС == 1 and mark.СЕМЕСТР == 1:
                rate += mark.mark
                count += 1
    count = max(1, count)
    global_count = max(1, global_count)
    school_rates.append(rate / count)
    g_school_rates.append(global_rate / global_count)
df = pd.DataFrame()
df["school"] = schools
df["rate"] = school_rates
df["global_rate"] = g_school_rates
st_info = pd.merge(st_info, df, left_on='school', right_on="school", how="left")
df

,school,rate,global_rate
0,39,3.987421,4.125439
1,69,4.000000,4.269608
2,33,4.351706,4.329181
3,64,4.364583,4.401695
4,189,3.733333,4.017778
...,...,...,...
220,213,4.000000,3.642857
221,127,4.200000,4.694444
222,221,4.800000,4.378378
223,72,4.000000,4.129032


In [155]:
agg_features = X_train_in.groupby('STD_ID')['mark'].agg(['mean','max','min','std','sum','count', np.median, np.ptp, np.average]).reset_index()

/home/ec2-user/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/ec2-user/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [156]:
hren = X_train_in.groupby(["НАПРАВЛЕНИЕ", "КУРС", "СЕМЕСТР", "АТТЕСТАЦИЯ", "ДИСЦИПЛИНА", "exam_year"]).mean()["mark"]
X_train_in = X_train_in.merge(hren, on=["НАПРАВЛЕНИЕ", "КУРС", "СЕМЕСТР", "АТТЕСТАЦИЯ", "ДИСЦИПЛИНА", "exam_year"], how="left")
difference = []
for i in zip(X_train_in["mark_x"], X_train_in["mark_y"]):
    difference.append(i[0] - i[1])
X_train_in["difference"] = difference
small_feature = X_train_in.groupby('STD_ID')['difference'].agg(['mean']).reset_index()
X_train_in = pd.merge(X_train_in,small_feature,on='STD_ID', how="left")
X_val = pd.merge(X_val,small_feature,on='STD_ID', how="left")

In [157]:
# new_info = st_info.groupby(st_info.index).first()
# new_info = new_info.fillna(0)
# new_info.head(1)

In [158]:
X_val = X_val.merge(st_info[["STD_ID", "age", "bablo", "learn", "school", "Пол", "rate", "global_rate"]], on="STD_ID", how="left")
X_train_in = X_train_in.merge(st_info[["STD_ID", "age", "bablo", "learn", "school", "Пол", "rate", "global_rate"]], how="left",left_on='STD_ID',right_on='STD_ID', validate = 'm:1')

In [159]:
X_train_in = pd.merge(X_train_in,agg_features,on='STD_ID')

In [160]:
X_train_in.head(3)

,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА,STD_LVL,big_group,small_group,...,global_rate,mean_y,max,min,std,sum,count,median,ptp,average
0,175711,38.03.02 Менеджмент,2018-2019,2.0,4.0,2,408,03,38,02,...,4.228495,4.181818,5,3,0.58849,92,22,4.0,2,4.181818
1,175711,38.03.02 Менеджмент,2017-2018,1.0,1.0,2,731,03,38,02,...,4.228495,4.181818,5,3,0.58849,92,22,4.0,2,4.181818
2,175711,38.03.02 Менеджмент,2017-2018,1.0,2.0,5,254,03,38,02,...,4.228495,4.181818,5,3,0.58849,92,22,4.0,2,4.181818


In [161]:
# stud_mean_mark = X_train_in.groupby("STD_ID").mean()["mark"]
# X_train_in.loc[:,"stud_mean"] = X_train_in.loc[:,"STD_ID"].map(stud_mean_mark)

In [162]:
X_train_in.shape

(105058, 31)

In [163]:
X_val['АТТЕСТАЦИЯ'].replace(['Дифференцированный зачет','Зачет', 'Курсовая работа', 'Курсовой проект','Экзамен'],[5,1,3,4,2],inplace=True)

In [164]:
X_val.head(3)

,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА,exam_year,mean,age,bablo,learn,school,Пол,rate,global_rate
0,163261,35.03.04 Агрономия,2018-2019,3.0,5.0,5,260,5,-0.092971,26,True,3,183,1.0,4.111842,4.254923
1,162990,23.03.03 Эксплуатация транспортно-технологичес...,2019-2020,4.0,7.0,2,1028,6,0.129593,22,True,6,39,1.0,3.987421,4.125439
2,192595,40.03.01 Юриспруденция,2018-2019,1.0,2.0,2,269,5,-0.181342,20,False,6,39,0.0,3.987421,4.125439


In [165]:
X_val = pd.merge(X_val, agg_features, on='STD_ID', how='left')

In [166]:
student_level = []
big_group = []
small_group = []

for i in X_val['НАПРАВЛЕНИЕ']:
    student_level.append(i.split('.')[1])
    big_group.append(i.split('.')[0])
    small_group.append(i.split('.')[2][:2])
    
X_val['STD_LVL'] = student_level
X_val['big_group'] = big_group
X_val['small_group'] = small_group

In [167]:
X_val.head(3)

,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА,exam_year,mean_x,age,...,min,std,sum,count,median,ptp,average,STD_LVL,big_group,small_group
0,163261,35.03.04 Агрономия,2018-2019,3.0,5.0,5,260,5,-0.092971,26,...,3.0,0.538958,86.0,21.0,4.0,2.0,4.095238,03,35,04
1,162990,23.03.03 Эксплуатация транспортно-технологичес...,2019-2020,4.0,7.0,2,1028,6,0.129593,22,...,3.0,0.772049,113.0,29.0,4.0,2.0,3.896552,03,23,03
2,192595,40.03.01 Юриспруденция,2018-2019,1.0,2.0,2,269,5,-0.181342,20,...,3.0,0.943880,43.0,11.0,4.0,2.0,3.909091,03,40,01


In [168]:
X_train_in.rename(columns={"КУРС": "course", "СЕМЕСТР": "semestr", "АТТЕСТАЦИЯ": "attestat", "ДИСЦИПЛИНА":"dis", "mean_x": "difference_mean", "mean_y": "mean", "Пол": "sex"}, errors="raise", inplace=True)
# X_test.rename(columns={"КУРС": "course", "СЕМЕСТР": "semestr", "АТТЕСТАЦИЯ": "attestat", "ДИСЦИПЛИНА":"dis", "mean_x": "difference_mean", "mean_y": "mean", "Пол": "sex"}, errors="raise", inplace=True)
X_val.rename(columns={"КУРС": "course", "СЕМЕСТР": "semestr", "АТТЕСТАЦИЯ": "attestat", "ДИСЦИПЛИНА":"dis", "mean_x": "difference_mean", "mean_y": "mean", "Пол": "sex"}, errors="raise", inplace=True)

In [169]:
from sklearn.model_selection import train_test_split

In [170]:
X_train, X_test, y_train, y_test = train_test_split(X_train_in,X_train_in["mark_x"], shuffle=True, test_size=0.01)

In [171]:
y_test.shape, y_train.shape

((1051,), (104007,))

In [172]:
from catboost import CatBoostRegressor, CatBoostClassifier

In [173]:
train_col = ["course","semestr", "attestat", "dis", "difference_mean", "std",
             "count", "max", "min", "STD_LVL", 'median', "sum",
             'ptp', 'average', 'big_group', 'small_group', "age", "bablo", "exam_year", "learn", "mean", "sex", "school", "rate"]
# X_train["STD_ID"]= X_train["STD_ID"].astype(int)
# X_test["STD_ID"]= X_test["STD_ID"].astype(int)


In [174]:
model = CatBoostClassifier(iterations=10000,  classes_count=6, depth = 10, learning_rate=0.3,)
# model = CatBoostRegressor(iterations=1000, learning_rate=0.05,)

In [175]:
model.fit(X_train[train_col],y_train)

Found only 3 unique classes in the data, but have defined 6 classes. Probably something is wrong with data.


0:	learn: 0.9507539	total: 70.2ms	remaining: 11m 41s
1:	learn: 0.8737841	total: 126ms	remaining: 10m 29s
2:	learn: 0.8264869	total: 182ms	remaining: 10m 5s
3:	learn: 0.7959912	total: 238ms	remaining: 9m 55s
4:	learn: 0.7734919	total: 293ms	remaining: 9m 45s
5:	learn: 0.7590222	total: 347ms	remaining: 9m 38s
6:	learn: 0.7473620	total: 402ms	remaining: 9m 34s
7:	learn: 0.7368696	total: 458ms	remaining: 9m 31s
8:	learn: 0.7320571	total: 471ms	remaining: 8m 42s
9:	learn: 0.7268405	total: 526ms	remaining: 8m 45s
10:	learn: 0.7208112	total: 581ms	remaining: 8m 47s
11:	learn: 0.7167329	total: 636ms	remaining: 8m 49s
12:	learn: 0.7130070	total: 690ms	remaining: 8m 50s
13:	learn: 0.7071800	total: 746ms	remaining: 8m 52s
14:	learn: 0.7046726	total: 801ms	remaining: 8m 53s
15:	learn: 0.7007952	total: 855ms	remaining: 8m 53s
16:	learn: 0.6986215	total: 910ms	remaining: 8m 54s
17:	learn: 0.6962848	total: 965ms	remaining: 8m 55s
18:	learn: 0.6940236	total: 1.02s	remaining: 8m 55s
19:	learn: 0.691487

161:	learn: 0.5264465	total: 9.34s	remaining: 9m 26s
162:	learn: 0.5257034	total: 9.4s	remaining: 9m 27s
163:	learn: 0.5251349	total: 9.46s	remaining: 9m 27s
164:	learn: 0.5244637	total: 9.52s	remaining: 9m 27s
165:	learn: 0.5238977	total: 9.58s	remaining: 9m 27s
166:	learn: 0.5232454	total: 9.64s	remaining: 9m 27s
167:	learn: 0.5225014	total: 9.7s	remaining: 9m 27s
168:	learn: 0.5215949	total: 9.77s	remaining: 9m 28s
169:	learn: 0.5205576	total: 9.84s	remaining: 9m 28s
170:	learn: 0.5200725	total: 9.9s	remaining: 9m 28s
171:	learn: 0.5195218	total: 9.96s	remaining: 9m 28s
172:	learn: 0.5190887	total: 10s	remaining: 9m 29s
173:	learn: 0.5178219	total: 10.1s	remaining: 9m 29s
174:	learn: 0.5172072	total: 10.1s	remaining: 9m 29s
175:	learn: 0.5168288	total: 10.2s	remaining: 9m 29s
176:	learn: 0.5161116	total: 10.3s	remaining: 9m 30s
177:	learn: 0.5154602	total: 10.3s	remaining: 9m 30s
178:	learn: 0.5151851	total: 10.4s	remaining: 9m 30s
179:	learn: 0.5145610	total: 10.5s	remaining: 9m 30

317:	learn: 0.4353254	total: 18.3s	remaining: 9m 16s
318:	learn: 0.4348428	total: 18.3s	remaining: 9m 15s
319:	learn: 0.4342446	total: 18.4s	remaining: 9m 15s
320:	learn: 0.4336887	total: 18.4s	remaining: 9m 15s
321:	learn: 0.4331532	total: 18.5s	remaining: 9m 15s
322:	learn: 0.4329193	total: 18.5s	remaining: 9m 15s
323:	learn: 0.4322756	total: 18.6s	remaining: 9m 14s
324:	learn: 0.4316469	total: 18.6s	remaining: 9m 14s
325:	learn: 0.4313010	total: 18.7s	remaining: 9m 14s
326:	learn: 0.4310824	total: 18.7s	remaining: 9m 14s
327:	learn: 0.4307490	total: 18.8s	remaining: 9m 13s
328:	learn: 0.4304502	total: 18.8s	remaining: 9m 13s
329:	learn: 0.4299165	total: 18.9s	remaining: 9m 13s
330:	learn: 0.4296584	total: 18.9s	remaining: 9m 12s
331:	learn: 0.4294007	total: 19s	remaining: 9m 12s
332:	learn: 0.4291676	total: 19s	remaining: 9m 12s
333:	learn: 0.4285903	total: 19.1s	remaining: 9m 12s
334:	learn: 0.4282078	total: 19.1s	remaining: 9m 11s
335:	learn: 0.4279763	total: 19.2s	remaining: 9m 1

475:	learn: 0.3698407	total: 27.6s	remaining: 9m 13s
476:	learn: 0.3693963	total: 27.7s	remaining: 9m 13s
477:	learn: 0.3691738	total: 27.8s	remaining: 9m 13s
478:	learn: 0.3688580	total: 27.8s	remaining: 9m 13s
479:	learn: 0.3685765	total: 27.9s	remaining: 9m 13s
480:	learn: 0.3681708	total: 28s	remaining: 9m 13s
481:	learn: 0.3677792	total: 28s	remaining: 9m 13s
482:	learn: 0.3674860	total: 28.1s	remaining: 9m 13s
483:	learn: 0.3669899	total: 28.2s	remaining: 9m 13s
484:	learn: 0.3667445	total: 28.2s	remaining: 9m 13s
485:	learn: 0.3664654	total: 28.3s	remaining: 9m 14s
486:	learn: 0.3660558	total: 28.4s	remaining: 9m 14s
487:	learn: 0.3655351	total: 28.4s	remaining: 9m 14s
488:	learn: 0.3652315	total: 28.5s	remaining: 9m 14s
489:	learn: 0.3649974	total: 28.6s	remaining: 9m 14s
490:	learn: 0.3647995	total: 28.6s	remaining: 9m 14s
491:	learn: 0.3644454	total: 28.7s	remaining: 9m 14s
492:	learn: 0.3641369	total: 28.8s	remaining: 9m 14s
493:	learn: 0.3638347	total: 28.8s	remaining: 9m 1

...

9338:	learn: 0.0371655	total: 9m 19s	remaining: 39.6s
9339:	learn: 0.0371640	total: 9m 19s	remaining: 39.5s
9340:	learn: 0.0371622	total: 9m 19s	remaining: 39.5s
9341:	learn: 0.0371585	total: 9m 19s	remaining: 39.4s
9342:	learn: 0.0371566	total: 9m 19s	remaining: 39.4s
9343:	learn: 0.0371555	total: 9m 19s	remaining: 39.3s
9344:	learn: 0.0371518	total: 9m 19s	remaining: 39.2s
9345:	learn: 0.0371488	total: 9m 19s	remaining: 39.2s
9346:	learn: 0.0371450	total: 9m 20s	remaining: 39.1s
9347:	learn: 0.0371426	total: 9m 20s	remaining: 39.1s
9348:	learn: 0.0371395	total: 9m 20s	remaining: 39s
9349:	learn: 0.0371390	total: 9m 20s	remaining: 38.9s
9350:	learn: 0.0371380	total: 9m 20s	remaining: 38.9s
9351:	learn: 0.0371352	total: 9m 20s	remaining: 38.8s
9352:	learn: 0.0371303	total: 9m 20s	remaining: 38.8s
9353:	learn: 0.0371288	total: 9m 20s	remaining: 38.7s
9354:	learn: 0.0371276	total: 9m 20s	remaining: 38.6s
9355:	learn: 0.0371252	total: 9m 20s	remaining: 38.6s
9356:	learn: 0.0371203	total: 

9494:	learn: 0.0367161	total: 9m 27s	remaining: 30.2s
9495:	learn: 0.0367150	total: 9m 27s	remaining: 30.1s
9496:	learn: 0.0367140	total: 9m 28s	remaining: 30.1s
9497:	learn: 0.0367097	total: 9m 28s	remaining: 30s
9498:	learn: 0.0367084	total: 9m 28s	remaining: 30s
9499:	learn: 0.0367046	total: 9m 28s	remaining: 29.9s
9500:	learn: 0.0367004	total: 9m 28s	remaining: 29.8s
9501:	learn: 0.0366989	total: 9m 28s	remaining: 29.8s
9502:	learn: 0.0366963	total: 9m 28s	remaining: 29.7s
9503:	learn: 0.0366947	total: 9m 28s	remaining: 29.7s
9504:	learn: 0.0366924	total: 9m 28s	remaining: 29.6s
9505:	learn: 0.0366909	total: 9m 28s	remaining: 29.5s
9506:	learn: 0.0366862	total: 9m 28s	remaining: 29.5s
9507:	learn: 0.0366833	total: 9m 28s	remaining: 29.4s
9508:	learn: 0.0366808	total: 9m 28s	remaining: 29.4s
9509:	learn: 0.0366791	total: 9m 28s	remaining: 29.3s
9510:	learn: 0.0366766	total: 9m 28s	remaining: 29.3s
9511:	learn: 0.0366747	total: 9m 29s	remaining: 29.2s
9512:	learn: 0.0366737	total: 9m

9649:	learn: 0.0363094	total: 9m 37s	remaining: 21s
9650:	learn: 0.0363080	total: 9m 38s	remaining: 20.9s
9651:	learn: 0.0363072	total: 9m 38s	remaining: 20.8s
9652:	learn: 0.0363052	total: 9m 38s	remaining: 20.8s
9653:	learn: 0.0363048	total: 9m 38s	remaining: 20.7s
9654:	learn: 0.0363018	total: 9m 38s	remaining: 20.7s
9655:	learn: 0.0362966	total: 9m 38s	remaining: 20.6s
9656:	learn: 0.0362944	total: 9m 38s	remaining: 20.5s
9657:	learn: 0.0362928	total: 9m 38s	remaining: 20.5s
9658:	learn: 0.0362897	total: 9m 38s	remaining: 20.4s
9659:	learn: 0.0362868	total: 9m 38s	remaining: 20.4s
9660:	learn: 0.0362833	total: 9m 38s	remaining: 20.3s
9661:	learn: 0.0362820	total: 9m 38s	remaining: 20.2s
9662:	learn: 0.0362793	total: 9m 38s	remaining: 20.2s
9663:	learn: 0.0362777	total: 9m 38s	remaining: 20.1s
9664:	learn: 0.0362749	total: 9m 38s	remaining: 20.1s
9665:	learn: 0.0362729	total: 9m 39s	remaining: 20s
9666:	learn: 0.0362701	total: 9m 39s	remaining: 19.9s
9667:	learn: 0.0362630	total: 9m

9802:	learn: 0.0359733	total: 9m 47s	remaining: 11.8s
9803:	learn: 0.0359667	total: 9m 47s	remaining: 11.7s
9804:	learn: 0.0359655	total: 9m 47s	remaining: 11.7s
9805:	learn: 0.0359608	total: 9m 47s	remaining: 11.6s
9806:	learn: 0.0359600	total: 9m 47s	remaining: 11.6s
9807:	learn: 0.0359583	total: 9m 47s	remaining: 11.5s
9808:	learn: 0.0359551	total: 9m 48s	remaining: 11.4s
9809:	learn: 0.0359513	total: 9m 48s	remaining: 11.4s
9810:	learn: 0.0359448	total: 9m 48s	remaining: 11.3s
9811:	learn: 0.0359440	total: 9m 48s	remaining: 11.3s
9812:	learn: 0.0359427	total: 9m 48s	remaining: 11.2s
9813:	learn: 0.0359406	total: 9m 48s	remaining: 11.2s
9814:	learn: 0.0359384	total: 9m 48s	remaining: 11.1s
9815:	learn: 0.0359315	total: 9m 48s	remaining: 11s
9816:	learn: 0.0359273	total: 9m 48s	remaining: 11s
9817:	learn: 0.0359247	total: 9m 48s	remaining: 10.9s
9818:	learn: 0.0359237	total: 9m 48s	remaining: 10.9s
9819:	learn: 0.0359199	total: 9m 48s	remaining: 10.8s
9820:	learn: 0.0359194	total: 9m

9958:	learn: 0.0355750	total: 9m 57s	remaining: 2.46s
9959:	learn: 0.0355733	total: 9m 57s	remaining: 2.4s
9960:	learn: 0.0355714	total: 9m 57s	remaining: 2.34s
9961:	learn: 0.0355657	total: 9m 57s	remaining: 2.28s
9962:	learn: 0.0355596	total: 9m 57s	remaining: 2.22s
9963:	learn: 0.0355573	total: 9m 57s	remaining: 2.16s
9964:	learn: 0.0355566	total: 9m 57s	remaining: 2.1s
9965:	learn: 0.0355533	total: 9m 57s	remaining: 2.04s
9966:	learn: 0.0355513	total: 9m 57s	remaining: 1.98s
9967:	learn: 0.0355478	total: 9m 57s	remaining: 1.92s
9968:	learn: 0.0355458	total: 9m 58s	remaining: 1.86s
9969:	learn: 0.0355446	total: 9m 58s	remaining: 1.8s
9970:	learn: 0.0355431	total: 9m 58s	remaining: 1.74s
9971:	learn: 0.0355394	total: 9m 58s	remaining: 1.68s
9972:	learn: 0.0355385	total: 9m 58s	remaining: 1.62s
9973:	learn: 0.0355376	total: 9m 58s	remaining: 1.56s
9974:	learn: 0.0355345	total: 9m 58s	remaining: 1.5s
9975:	learn: 0.0355312	total: 9m 58s	remaining: 1.44s
9976:	learn: 0.0355265	total: 9m

In [176]:
mean_absolute_error(y_test, model.predict(X_test[train_col])), r2_score(y_test, model.predict(X_test[train_col]))

(0.3653663177925785, 0.29565841180745867)

In [177]:
y_pred = pd.read_csv("sample_submission.csv",index_col=0)
y_pred["mark"] = model.predict(X_val[train_col])

In [178]:
y_pred[y_pred["mark"]<3]

,mark
index,


In [179]:
#y_pred[y_pred["mark"]<3] = 3

In [180]:
#print(model.predict(X_val[train_col]).shape, pd.read_csv("sample_submission.csv",index_col=0).shape)

In [181]:
import time
import os

current_timestamp = int(time.time())
submission_path = 'Submissions/{}.csv'.format(current_timestamp)

if not os.path.exists('Submissions'):
    os.makedirs('Submissions')

print(submission_path)
y_pred.to_csv(submission_path, index=True)

Submissions/1585426942.csv
